In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

讀取的檔案，已經涵蓋了新聞的正負面屬性，今天我們希望找出每篇新聞的關鍵字  
因此除了要做結巴切詞之外，同時也希望得到tf-idf等等資訊  
不過再這之前，我們希望對新聞資訊做一些處理(處理亂碼、標點符號、停用詞等等)  
更進階一點，為了讓結巴切的準確一些，我們可以新增字典，讓結巴沒看過的詞彙也分的出來   

In [46]:
LARGAN_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/大立光正負新聞.csv')
Foxconn_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/鴻海正負新聞.csv')
TSMC_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/台積電正負新聞.csv')


### 正則表達式把文章弄得好看一點

## 切割出重要詞彙

In [47]:
def cut(df):

    upnews = df[df['status']==1]
    downnews = df[df['status']==-1]
    
    positive_token = []
    for goodnews in upnews['CONTENT']:
        goodnews = re.sub(r'[。，（）_?\d]','',goodnews)
        #這邊想再加一行過濾亂碼的條件，但是還沒搞懂
        seg_list = jieba.cut_for_search(goodnews)
        token = ' '.join(seg_list)
        positive_token.append(token)
    
    count_vec=CountVectorizer() 
    good_news_count = count_vec.fit_transform(positive_token) 
    good_news_count = pd.DataFrame(good_news_count.toarray(),columns = count_vec.get_feature_names())
    #這兩行如果不做的話，concat起來會因為index不同而出現NaN
    good_news_count.reset_index(drop=True, inplace=True)  
    upnews.reset_index(drop=True, inplace=True)

    positive_news = pd.concat([upnews,good_news_count],axis = 1)

    
    negative_token = []
    for badnews in downnews['CONTENT']:
        badnews = re.sub(r'[。，（）_?\d]','',badnews)
        seg_list = jieba.cut_for_search(badnews)
        token = ' '.join(seg_list)
        negative_token.append(token)
    
    count_vec=CountVectorizer() 
    bad_news_count = count_vec.fit_transform(negative_token) 
    bad_news_count = pd.DataFrame(bad_news_count.toarray(),columns = count_vec.get_feature_names())
    
    bad_news_count.reset_index(drop=True, inplace=True)  
    downnews.reset_index(drop=True, inplace=True)
    
    
    negative_news = pd.concat([downnews,bad_news_count],axis = 1)
    
    return positive_news,negative_news

In [48]:
positive_Foxconn_NEWS = cut(Foxconn_NEWS)[0]
negative_Foxconn_NEWS = cut(Foxconn_NEWS)[1]
positive_LARGAN_NEWS = cut(LARGAN_NEWS)[0]
negative_LARGAN_NEWS = cut(LARGAN_NEWS)[1]
positive_TSMC_NEWS = cut(TSMC_NEWS)[0]
negative_TSMC_NEWS = cut(TSMC_NEWS)[1]

In [49]:
positive_Foxconn_NEWS

,TITLE,TIME,DESCRIPTION,CONTENT,year,month,day,hour,status,abf,...,龍虎斑,龍鎮,龍頭,龍黑,龐大,龐大內,龐大勞工,龐大銷量,龜山,龜山區
0,台股量能 縮至千億元,2019-10-21,新台幣昨（21）日續強，外資連六個交易日買超台股，持續加碼鴻海、大立光，發揮穩盤功效收小紅，...,台股示意圖。 歐新社 分享 facebook 新台幣昨（21）日續強，外資連六個...,2019,10,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,選情初探／北市立委3選區 藍綠白三帥對決 話題性十足,2019-10-21,台北市立委選舉第三選區（中山、北松山區）由國民黨現任立委蔣萬安，對上民進黨徵召的吳怡農和台灣...,台北市立委選舉第三選區（中山、北松山區）由國民黨現任立委蔣萬安，對上民進黨徵召的吳怡農和台灣...,2019,10,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,FII搶市 大秀專業雲,2019-10-21,鴻海集團近期積極開拓新專業雲端應用，鴻海子公司工業富聯（FII）昨（21）日也發布新聞稿提到...,工業富聯（FII）搶攻專業雲端應用，圖為FII的深圳關燈工廠。 網路照片 分享 face...,2019,10,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,12檔外資積極作多 本周成長力道延續,2019-10-21,台股在KD高檔鈍化狀態下維持高檔震盪，法人表示，上周外資積極作多的鴻海（2317）、聯電（2...,台股在KD高檔鈍化狀態下維持高檔震盪，法人表示，上周外資積極作多的鴻海（2317）、聯電（2...,2019,10,21,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,精誠取得傑克斯德Red Hat雲平台資源,2019-10-21,資訊服務商精誠（6214）今（21）日宣布延攬擁有雲端領域技術的台灣技術團隊傑克斯德資訊科技...,資訊服務商精誠（6214）今（21）日宣布延攬擁有雲端領域技術的台灣技術團隊傑克斯德資訊科技...,2019,10,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,緯創、台光電 外資叫好,2019-03-27,緯創（3231）公布財報，令外資驚豔，?豐證券出具報告看好緯創資料中心業務加速成長，目標價上...,緯創（3231）公布財報，令外資驚豔，?豐證券出具報告看好緯創資料中心業務加速成長，目標價上...,2019,3,27,0,1,0,...,0,0,0,0,0,0,0,0,0,0
415,係金A！ 加郭董好友 一天飆破4萬人,2019-03-27,鴻海集團總裁郭台銘LINE@帳號昨啟用，用詞有趣的「昨晚郭董LINE我，一直拜託我加他好友」...,鴻海集團總裁郭台銘LINE帳號昨開張，台中市很多人轉傳郭的「加好友」訊息，一起當「郭台銘的好...,2019,3,27,0,1,0,...,0,0,0,0,0,0,0,0,0,0
416,今日台股盤前重點掃描,2019-03-27,一、美股上漲 金融股隨公債殖利率回穩；蘋果下跌拖累大盤\n美國股市周二上漲，金融股擺脫連五日...,今日台股盤前重點掃描。 圖／聯合報系資料照片 分享 facebook 一、美股上...,2019,3,27,0,1,0,...,0,0,1,0,1,0,0,0,0,0
417,緯創攜鴻海、索尼等 布局新世代智能攝像系統生態系,2019-03-27,緯創今（27）日表示，該公司除了專精於智慧手機、筆電、伺服器與數位醫療等ICT產品及服務代工...,緯創今（27）日表示，該公司除了專精於智慧手機、筆電、伺服器與數位醫療等ICT產品及服務代工...,2019,3,27,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
positive_Foxconn_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/鴻海正面新聞.csv',index = None)
negative_Foxconn_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/鴻海負面新聞.csv',index = None)
positive_LARGAN_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/大立光正面新聞.csv',index = None)
negative_LARGAN_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/大立光負面新聞.csv',index = None)
positive_TSMC_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/台積電正面新聞.csv',index = None)
negative_TSMC_NEWS.to_csv('/Users/liaoshousan/Desktop/RLSD_PROJECT-StockPrice Prediction/提取出正負面的新聞/台積電負面新聞.csv',index = None)

